In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.datasets import eegbci
from mne.preprocessing import ICA
#mne.viz.set_3d_backend("notebook")
%matplotlib qt
%gui qt

In [ ]:
samprate = 1024
references = ["M1", "M2"] 
hifilt = 45
lowfilt = 0.5 
externos = ["EOG"] 

path_suj_MF = 'C:/Users/Sebastian/Documents/NeuroZenProcessing/ParticipantsSet/Sujeto3_MF.set'
path_suj_R = 'C:/Users/Sebastian/Documents/NeuroZenProcessing/ParticipantsSet/Sujeto3_R.set'
path_guardar_suj = 'C:/Users/Sebastian/Documents/NeuroZenProcessing/Preprocessed/' 
name_guardar_suj_MF = 'Sujeto3_MF_processed.mat'
name_guardar_suj_R = 'Sujeto3_R_processed.mat'

In [ ]:
raw_data_MF = mne.io.read_raw_eeglab(path_suj_MF, eog = externos, preload=True)
raw_data_R = mne.io.read_raw_eeglab(path_suj_R, eog = externos, preload=True)

In [ ]:
channel_names = raw_data_MF.ch_names
print(channel_names)
channel_names = [x for x in channel_names if x not in externos]

montage = mne.channels.make_standard_montage('standard_1020')

ind = [i for (i, channel) in enumerate(montage.ch_names) if channel in channel_names]
montage_new = montage.copy()
# Keep only the desired channels
montage_new.ch_names = [montage.ch_names[x] for x in ind]
kept_channel_info = [montage.dig[x+3] for x in ind]
# Keep the first three rows as they are the fiducial points information
montage_new.dig = montage.dig[0:3]+kept_channel_info
montage_new.plot(sphere=(0, 0.02, 0, 0.095))

In [ ]:
raw_data_MF.set_montage(montage_new)
raw_data_R.set_montage(montage_new)

#print(raw_data_MF.ch_names)

#drop eog channel
raw_data_MF.drop_channels(externos)
raw_data_R.drop_channels(externos)

#set references
raw_data_MF.set_eeg_reference(ref_channels=references)
raw_data_R.set_eeg_reference(ref_channels=references)
#print(raw_data_MF.info['chs'])

In [ ]:
raw_data_MF.filter(l_freq=lowfilt, h_freq=hifilt)
raw_data_R.filter(l_freq=lowfilt, h_freq=hifilt)

In [ ]:
raw_data_MF.resample(1024, npad='auto')
raw_data_R.resample(1024, npad='auto')

In [ ]:
#check bad channels

raw_data_MF.plot(scalings = dict(eeg=200e-6))

In [ ]:
raw_data.info['bads'] = [] #note the bad channels
picks = mne.pick_types(raw_data.info, exclude='bads')


In [ ]:
raw_data.interpolate_bads(reset_bads=True)
raw_data.plot(scalings=dict(eeg=200e-6))

In [ ]:
num_components = 20
ica = ICA(n_components=num_components, method='fastica')
ica.fit(raw_data)

In [ ]:
ica.plot_components()

In [ ]:
#raw_data.plot(n_channels=63, scalings=dict(eeg=100e-6)) 


In [ ]:
#ica.plot_properties(raw_data, picks=); # This exact component number probably won't work if you recompute ICA

In [ ]:
#ica.plot_overlay(raw, exclude=[])

In [ ]:
ica.exclude = []
eog_indices, eog_scores = ica.find_bads_eog(raw_data)
ica.exclude = eog_indices

In [ ]:
#barplot of ICA component "EOG match" scores
#ica.plot_scores(eog_scores)

#plot diagnostics
#ica.plot_properties(raw_data, picks=eog_indices)

#plot ICs applied to raw data, with EOG matches highlighted
#ica.plot_sources(raw_data, show_scrollbars=False)


In [ ]:
ica.apply(raw_data)
raw_data.plot(scalings=dict(eeg=200e-6))

In [ ]:
print(raw_data.info["projs"])

In [ ]:
mne.viz.plot_raw_psd(raw_data, fmax=18, picks=['eeg'])

In [ ]:
#mne.viz.plot_raw_psd_topo(raw_data, fmax=18)

In [ ]:
spectrum = raw_data.compute_psd()

bands = {'Delta (0-4 Hz)': (0, 4), 'Theta (4-8 Hz)': (4, 8),
         'Alpha (8-12 Hz)': (8, 12), 'Beta (12-30 Hz)': (12, 30)}

spectrum.plot_topomap(bands= bands,sphere='eeglab', ch_type='eeg', cmap='jet')